<a href="https://colab.research.google.com/github/recervictory/LearingPython/blob/master/09_Pandas_Data_Wrangling_Join_Combine_and_Reshape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Wrangling: Join, Combine, and Reshape

In [18]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## 0. Indexing with a DataFrame’s columns

In [19]:
frame = pd.DataFrame({'roll': range(7), 
                      'marks': range(7, 0, -1), 
                      'group': ['one', 'one', 'one', 'two', 'two', 'two', 'two'], 
                      'id': [0, 1, 2, 0, 1, 2, 3]})

frame

,roll,marks,group,id
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [20]:
frame.set_index(['roll'])

,marks,group,id
roll,,,
0,7,one,0
1,6,one,1
2,5,one,2
3,4,two,0
4,3,two,1
5,2,two,2
6,1,two,3


In [21]:
# seting new index
frameNew = frame.set_index(['group', 'id'])
frameNew

roll  marks
group id             
one   0      0      7
      1      1      6
      2      2      5
two   0      3      4
      1      4      3
      2      5      2
      3      6      1

In [22]:
# Not removing the original column
frame.set_index(['group', 'id'], drop=False)

roll  marks group  id
group id                       
one   0      0      7   one   0
      1      1      6   one   1
      2      2      5   one   2
two   0      3      4   two   0
      1      4      3   two   1
      2      5      2   two   2
      3      6      1   two   3

## 1. Hierarchical Indexing
Hierarchical indexing is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis. Somewhat abstractly, it provides a way for you to work with higher dimensional data in a lower dimensional form.

Hierarchical indexing plays an important role in reshaping data and **group-based** operations like forming a **pivot table**. For example, you could rearrange the data into a DataFrame using its unstack method:

In [23]:
# Hierarchical Indexing
data =  pd.Series(np.random.randn(9), \
        index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'], \
        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.174283
   2    0.617997
   3   -0.012588
b  1    1.035543
   3    0.757337
c  1    0.219159
   2    0.256418
d  2   -0.151269
   3   -0.182112
dtype: float64

In [24]:
# Find Out Index
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [25]:
data['b']

1    1.035543
3    0.757337
dtype: float64

In [26]:
data['b':'c']

b  1    1.035543
   3    0.757337
c  1    0.219159
   2    0.256418
dtype: float64

In [27]:
data.loc[['b', 'd']]

b  1    1.035543
   3    0.757337
d  2   -0.151269
   3   -0.182112
dtype: float64

Hierarchical indexing plays an important role in reshaping data and group-based
operations like forming a `pivot table`. For example, you could rearrange the data into a DataFrame using its `unstack method`:

In [28]:
data.unstack()

,1,2,3
a,-0.174283,0.617997,-0.012588
b,1.035543,NaN,0.757337
c,0.219159,0.256418,NaN
d,NaN,-0.151269,-0.182112


In [29]:
# The inverse operation of unstack is stack
data.unstack().stack()

a  1   -0.174283
   2    0.617997
   3   -0.012588
b  1    1.035543
   3    0.757337
c  1    0.219159
   2    0.256418
d  2   -0.151269
   3   -0.182112
dtype: float64

In [30]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)), 
                     index=[['jan', 'jan', 'feb', 'feb'], [2011, 2012, 2011, 2012]],
                     columns=[['Kolkata', 'Kolkata', 'Delhi'],
                              ['Green', 'Red', 'Green']])

frame

Kolkata     Delhi
           Green Red Green
jan 2011       0   1     2
    2012       3   4     5
feb 2011       6   7     8
    2012       9  10    11

In [31]:
# Show index key
frame.index.names = ['month', 'year']
frame

Kolkata     Delhi
             Green Red Green
month year                  
jan   2011       0   1     2
      2012       3   4     5
feb   2011       6   7     8
      2012       9  10    11

In [32]:
# Show column names
frame.columns.names = ['city', 'color']

In [33]:
frame

city       Kolkata     Delhi
color        Green Red Green
month year                  
jan   2011       0   1     2
      2012       3   4     5
feb   2011       6   7     8
      2012       9  10    11

In [34]:
# Reseting the index
frameNew.reset_index()

,group,id,roll,marks
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 2. Reordering and Sorting Levels

In [35]:
frame.swaplevel('year', 'month') 

city       Kolkata     Delhi
color        Green Red Green
year month                  
2011 jan         0   1     2
2012 jan         3   4     5
2011 feb         6   7     8
2012 feb         9  10    11

`sort_index`, on the other hand, sorts the data using only the values in a single level. When swapping levels, it’s not uncommon to also use sort_index so that the result is lexicographically sorted by the indicated level

In [36]:
frame.sort_index(level=0)

city       Kolkata     Delhi
color        Green Red Green
month year                  
feb   2011       6   7     8
      2012       9  10    11
jan   2011       0   1     2
      2012       3   4     5

In [37]:
frame

city       Kolkata     Delhi
color        Green Red Green
month year                  
jan   2011       0   1     2
      2012       3   4     5
feb   2011       6   7     8
      2012       9  10    11

## 3. Summary Statistics by Level

Many descriptive and summary statistics on DataFrame and `Series` have a level
option in which you can specify the level you want to `aggregate` by on a particular axis. 

In [38]:
frame.mean(level='month')

city  Kolkata      Delhi
color   Green  Red Green
month                   
jan       1.5  2.5   3.5
feb       7.5  8.5   9.5

In [39]:
frame.sum(level='city', axis=1) # column wise

city        Kolkata  Delhi
month year                
jan   2011        1      2
      2012        7      5
feb   2011       13      8
      2012       19     11

## 4. Combining and Merging Datasets

Data contained in pandas objects can be combined together in a number of ways:
- `pandas.merge` connects rows in DataFrames based on one or more keys. This
will be familiar to users of SQL or other relational databases, as it implements
database join operations.
-  `pandas.concat` concatenates or “stacks” together objects along an axis.
-  The `combine_first` instance method enables splicing together overlapping data to fill in missing values in one object with values from another.



### Database-Style DataFrame Joins
Merge or join operations combine datasets by linking rows using one or more keys. These operations are central to relational databases (e.g., SQL-based). The merge function in pandas is the main entry point for using these algorithms on your data.

In [40]:
# 1st Dataframe
df1 = pd.DataFrame({'name': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                    'math': range(7)})
df1

,name,math
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [41]:
# 2nd Dataframe
df2 = pd.DataFrame({'name': [ 'b', 'd','c'], 'bio': range(3)})
df2

,name,bio
0,b,0
1,d,1
2,c,2


In [42]:
pd.merge(df1,df2)


,name,math,bio
0,b,0,0
1,b,1,0
2,b,6,0
3,c,3,2


##### Note that I didn’t specify which column to join on. If that information is not specified, merge uses the **overlapping column names** as the keys. It’s a good practice to specify explicitly, though:

In [43]:
pd.merge(df1, df2, on='name') # merge on the basis of name column

,name,math,bio
0,b,0,0
1,b,1,0
2,b,6,0
3,c,3,2


In [44]:
# If the column names are different in each object, you can specify them separately:
df3 = pd.DataFrame({'lname': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 
                    'math': range(7)})
df4 = pd.DataFrame({'rname': ['a', 'b', 'd'], 'bio': range(3)})


In [45]:
pd.merge(df3, df4, left_on='lname', right_on='rname')

,lname,math,rname,bio
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


## Relational Algebra

The behavior implemented in ``pd.merge()`` is a subset of what is known as *relational algebra*, which is a formal set of rules for manipulating relational data, and forms the conceptual foundation of operations available in most databases.
The strength of the relational algebra approach is that it proposes several primitive operations, which become the building blocks of more complicated operations on any dataset.
With this lexicon of fundamental operations implemented efficiently in a database or other program, a wide range of fairly complicated composite operations can be performed.

Pandas implements several of these fundamental building-blocks in the ``pd.merge()`` function and the related ``join()`` method of ``Series`` and ``Dataframe``s.
As we will see, these let you efficiently link data from different sources.

### Type of JOIN in DataFrame
![join](https://cdn.mindmajix.com/blog/images/db-01_2119.png "Data Frame Join")

By default merge does an 'inner' join; the keys in the result are the intersec‐
tion, or the common set found in both tables. Other possible options are 'left',
'right', and 'outer'. The outer join takes the union of the keys, combining the
effect of applying both left and right joins:

### Different join types with how argument
- 'inner' Use only the key combinations observed in both tables
- 'left' Use all key combinations found in the left table
- 'right' Use all key combinations found in the right table
- 'output' Use all key combinations observed in both tables together



In [46]:
pd.merge(df1, df2, how='outer')

,name,math,bio
0,b,0.0,0.0
1,b,1.0,0.0
2,b,6.0,0.0
3,a,2.0,NaN
4,a,4.0,NaN
5,a,5.0,NaN
6,c,3.0,2.0
7,d,NaN,1.0


### Merging on Index
In some cases, the merge key(s) in a DataFrame will be found in its index. In this case, you can pass left_index=True or right_index=True (or both) to indicate that the index should be used as the merge key

In [47]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [48]:
# Using Outer Join 
pd.merge(left1, right1, left_on='key', right_index=True, how='outer') # if it is on index right/left_index =  True
# column right_on 

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [49]:
lefth = pd.DataFrame({'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                      'year': [2000, 2001, 2002, 2001, 2002], 
                      'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio','Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [50]:
lefth

,state,year,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [51]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [52]:
pd.merge(lefth, righth, left_on=['state', 'year'], right_index=True)

,state,year,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [53]:
pd.merge(lefth, righth, left_on=['state', 'year'], right_index=True,how="outer")

,state,year,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


## Categories of Joins

The ``pd.merge()`` function implements a number of types of joins: the *one-to-one*, *many-to-one*, and *many-to-many* joins.
All three types of joins are accessed via an identical call to the ``pd.merge()`` interface; the type of join performed depends on the form of the input data.
Here we will show simple examples of the three types of merges, and discuss detailed options further below.

### One-to-one joins

Perhaps the simplest type of merge expresion is the one-to-one join, which is in many ways very similar to the column-wise concatenation.
As a concrete example, consider the following two ``DataFrames`` which contain information on several employees in a company:

In [54]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


To combine this information into a single ``DataFrame``, we can use the ``pd.merge()`` function:

In [55]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


The ``pd.merge()`` function recognizes that each ``DataFrame`` has an "employee" column, and automatically joins using this column as a key.
The result of the merge is a new ``DataFrame`` that combines the information from the two inputs.
Notice that the order of entries in each column is not necessarily maintained: in this case, the order of the "employee" column differs between ``df1`` and ``df2``, and the ``pd.merge()`` function correctly accounts for this.
Additionally, keep in mind that the merge in general discards the index, except in the special case of merges by index (see the ``left_index`` and ``right_index`` keywords, discussed momentarily).

Many-to-one joins are joins in which one of the two key columns contains duplicate entries.
For the many-to-one case, the resulting ``DataFrame`` will preserve those duplicate entries as appropriate.
Consider the following example of a many-to-one join:

In [56]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

The resulting ``DataFrame`` has an aditional column with the "supervisor" information, where the information is repeated in one or more locations as required by the inputs.

### Many-to-many joins
Many-to-many joins are a bit confusing conceptually, but are nevertheless well defined.
If the key column in both the left and right array contains duplicates, then the result is a many-to-many merge.
This will be perhaps most clear with a concrete example.
Consider the following, where we have a ``DataFrame`` showing one or more skills associated with a particular group.
By performing a many-to-many join, we can recover the skills associated with any individual person:

In [57]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


These three types of joins can be used with other Pandas tools to implement a wide array of functionality.
But in practice, datasets are rarely as clean as the one we're working with here.
In the following section we'll consider some of the options provided by ``pd.merge()`` that enable you to tune how the join operations work.

## Specification of the Merge Key
We've already seen the default behavior of ``pd.merge()``: it looks for one or more matching column names between the two inputs, and uses this as the key.
However, often the column names will not match so nicely, and ``pd.merge()`` provides a variety of options for handling this.

### The ``on`` keyword

Most simply, you can explicitly specify the name of the key column using the ``on`` keyword, which takes a column name or a list of column names:

In [58]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

This option works only if both the left and right ``DataFrame``s have the specified column name.

### The ``left_on`` and ``right_on`` keywords

At times you may wish to merge two datasets with different column names; for example, we may have a dataset in which the employee name is labeled as "name" rather than "employee".
In this case, we can use the ``left_on`` and ``right_on`` keywords to specify the two column names:

In [59]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

The result has a redundant column that we can drop if desired–for example, by using the ``drop()`` method of ``DataFrame``s:

In [60]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### The ``left_index`` and ``right_index`` keywords

Sometimes, rather than merging on a column, you would instead like to merge on an index.
For example, your data might look like this:

In [61]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


You can use the index as the key for merging by specifying the ``left_index`` and/or ``right_index`` flags in ``pd.merge()``:

In [62]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


If you'd like to mix indices and columns, you can combine ``left_index`` with ``right_on`` or ``left_on`` with ``right_index`` to get the desired behavior:

In [63]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

## Specifying Set Arithmetic for Joins

In [64]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Here we have merged two datasets that have only a single "name" entry in common: Mary.
By default, the result contains the *intersection* of the two sets of inputs; this is what is known as an *inner join*.
We can specify this explicitly using the ``how`` keyword, which defaults to ``"inner"``:

In [65]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Other options for the ``how`` keyword are ``'outer'``, ``'left'``, and ``'right'``.
An *outer join* returns a join over the union of the input columns, and fills in all missing values with NAs:

In [66]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

The *left join* and *right join* return joins over the left entries and right entries, respectively.
For example:

In [67]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

The output rows now correspond to the entries in the left input. Using
``how='right'`` works in a similar manner.

All of these options can be applied straightforwardly to any of the preceding join types.

## Overlapping Column Names: The ``suffixes`` Keyword
Finally, you may end up in a case where your two input ``DataFrame``s have conflicting column names.
Consider this example:

In [68]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Because the output would have two conflicting column names, the merge function automatically appends a suffix ``_x`` or ``_y`` to make the output columns unique.
If these defaults are inappropriate, it is possible to specify a custom suffix using the ``suffixes`` keyword:

In [69]:
#@title
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2